# 🧬 AlphaFold Protein Structure Comparison

## Real AlphaFold API Analysis

Comparing:
- **High Confidence**: Human Ubiquitin (P0CG48) - Well-folded protein
- **Low Confidence**: Human Alpha-synuclein (P37840) - Intrinsically disordered protein (IDP)

**Data Source**: Real AlphaFold Database API (alphafold.ebi.ac.uk)

In [1]:
# Enable inline plotting for Jupyter
%matplotlib inline

import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import tempfile
import os
import json

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports complete")

✅ Imports complete


## Configuration

In [2]:
ALPHAFOLD_API_BASE = "https://alphafold.ebi.ac.uk/api"
ALPHAFOLD_FILES_BASE = "https://alphafold.ebi.ac.uk/files"

HIGH_CONFIDENCE_PROTEIN = {
    'name': 'Human Ubiquitin',
    'uniprot_id': 'P0CG48',
    'expected_mean_plddt': '>90',
}

LOW_CONFIDENCE_PROTEIN = {
    'name': 'Human Alpha-Synuclein',
    'uniprot_id': 'P37840',
    'expected_mean_plddt': '<70',
}

print("📊 Proteins configured:")
print(f"  1. {HIGH_CONFIDENCE_PROTEIN['name']} ({HIGH_CONFIDENCE_PROTEIN['uniprot_id']})")
print(f"  2. {LOW_CONFIDENCE_PROTEIN['name']} ({LOW_CONFIDENCE_PROTEIN['uniprot_id']})")

📊 Proteins configured:
  1. Human Ubiquitin (P0CG48)
  2. Human Alpha-Synuclein (P37840)


## Functions

In [3]:
def get_alphafold_metadata(uniprot_id: str) -> Dict:
    """Get metadata from AlphaFold API"""
    url = f"{ALPHAFOLD_API_BASE}/prediction/{uniprot_id}"
    print(f"🔍 Querying API: {uniprot_id}")
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    data = response.json()
    metadata = data[0] if isinstance(data, list) else data
    print(f"   ✅ Found: {metadata.get('gene', 'N/A')} ({metadata.get('uniprotEnd', 'N/A')} residues)")
    return metadata

def download_alphafold_structure(uniprot_id: str, version: int = 4) -> str:
    """Download PDB file from AlphaFold"""
    pdb_filename = f"AF-{uniprot_id}-F1-model_v{version}.pdb"
    pdb_url = f"{ALPHAFOLD_FILES_BASE}/{pdb_filename}"
    print(f"📥 Downloading v{version}...", end=" ")
    response = requests.get(pdb_url, timeout=60)
    response.raise_for_status()
    temp_dir = tempfile.gettempdir()
    local_pdb_path = os.path.join(temp_dir, pdb_filename)
    with open(local_pdb_path, 'w') as f:
        f.write(response.text)
    print(f"✅ {len(response.text):,} bytes")
    return local_pdb_path

def extract_plddt_from_pdb(pdb_file: str) -> Tuple[np.ndarray, List[int]]:
    """Extract pLDDT scores from PDB B-factor column"""
    plddt_scores = []
    residue_numbers = []
    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('ATOM') and ' CA ' in line:
                res_num = int(line[22:26].strip())
                bfactor = float(line[60:66].strip())
                plddt_scores.append(bfactor)
                residue_numbers.append(res_num)
    return np.array(plddt_scores), residue_numbers

def calculate_plddt_statistics(plddt: np.ndarray) -> Dict:
    """Calculate statistics - all Python native types for JSON"""
    stats = {
        'mean': float(np.mean(plddt)),
        'median': float(np.median(plddt)),
        'std': float(np.std(plddt)),
        'min': float(np.min(plddt)),
        'max': float(np.max(plddt)),
        'q25': float(np.percentile(plddt, 25)),
        'q75': float(np.percentile(plddt, 75)),
        'very_high_conf': float(np.sum(plddt > 90) / len(plddt) * 100),
        'high_conf': float(np.sum((plddt > 70) & (plddt <= 90)) / len(plddt) * 100),
        'medium_conf': float(np.sum((plddt > 50) & (plddt <= 70)) / len(plddt) * 100),
        'low_conf': float(np.sum(plddt <= 50) / len(plddt) * 100),
    }
    return stats

print("✅ Functions defined")

✅ Functions defined


## Analyze Proteins

In [4]:
def analyze_protein(protein_info: Dict) -> Dict:
    """Complete analysis pipeline for one protein"""
    print(f"\n{'='*70}")
    print(f"🔬 ANALYZING: {protein_info['name']}")
    print(f"{'='*70}")
    
    # Get metadata and download structure
    metadata = get_alphafold_metadata(protein_info['uniprot_id'])
    version = metadata.get('latestVersion', 4)
    pdb_file = download_alphafold_structure(protein_info['uniprot_id'], version)
    
    # Extract pLDDT and calculate stats
    print("📊 Extracting pLDDT scores...", end=" ")
    plddt, residue_numbers = extract_plddt_from_pdb(pdb_file)
    print(f"✅ {len(plddt)} residues")
    
    stats = calculate_plddt_statistics(plddt)
    
    # Determine confidence level
    if stats['mean'] > 90:
        confidence = 'VERY HIGH'
    elif stats['mean'] > 70:
        confidence = 'HIGH'
    elif stats['mean'] > 50:
        confidence = 'MEDIUM'
    else:
        confidence = 'LOW'
    
    # Print results
    print(f"\n📊 RESULTS:")
    print(f"   Mean pLDDT:    {stats['mean']:.1f}")
    print(f"   Std Dev:       {stats['std']:.1f}")
    print(f"   Range:         {stats['min']:.1f} - {stats['max']:.1f}")
    print(f"   Confidence:    {confidence}")
    print(f"   Very High %:   {stats['very_high_conf']:.1f}%")
    
    return {
        'protein_info': protein_info,
        'metadata': metadata,
        'plddt': plddt,
        'residue_numbers': residue_numbers,
        'statistics': stats,
        'confidence': confidence,
    }

# Run analysis
print("🚀 Starting AlphaFold Analysis\n")
high_conf = analyze_protein(HIGH_CONFIDENCE_PROTEIN)
low_conf = analyze_protein(LOW_CONFIDENCE_PROTEIN)

print("\n✅ Both proteins analyzed successfully!")

🚀 Starting AlphaFold Analysis


🔬 ANALYZING: Human Ubiquitin
🔍 Querying API: P0CG48
   ✅ Found: UBC (685 residues)
📥 Downloading v6... ✅ 446,957 bytes
📊 Extracting pLDDT scores... ✅ 685 residues

📊 RESULTS:
   Mean pLDDT:    88.6
   Std Dev:       8.4
   Range:         33.4 - 97.1
   Confidence:    HIGH
   Very High %:   56.1%

🔬 ANALYZING: Human Alpha-Synuclein
🔍 Querying API: P37840
   ✅ Found: SNCA (140 residues)
📥 Downloading v6... ✅ 86,831 bytes
📊 Extracting pLDDT scores... ✅ 140 residues

📊 RESULTS:
   Mean pLDDT:    75.2
   Std Dev:       18.3
   Range:         39.5 - 94.6
   Confidence:    HIGH
   Very High %:   28.6%

✅ Both proteins analyzed successfully!


## Comparison Summary

In [ ]:
# Calculate differences
delta_mean = high_conf['statistics']['mean'] - low_conf['statistics']['mean']
ratio = high_conf['statistics']['mean'] / low_conf['statistics']['mean']

print("="*70)
print("📊 COMPARISON SUMMARY")
print("="*70)
print(f"\n1️⃣  {HIGH_CONFIDENCE_PROTEIN['name']}:")
print(f"    Mean pLDDT: {high_conf['statistics']['mean']:.1f}")
print(f"    Confidence: {high_conf['confidence']}")

print(f"\n2️⃣  {LOW_CONFIDENCE_PROTEIN['name']}:")
print(f"    Mean pLDDT: {low_conf['statistics']['mean']:.1f}")
print(f"    Confidence: {low_conf['confidence']}")

print(f"\n📈 KEY DIFFERENCES:")
print(f"    Δ pLDDT:  {delta_mean:.1f} points")
print(f"    Ratio:    {ratio:.2f}x")

print(f"\n💡 INTERPRETATION:")
if delta_mean > 20:
    print(f"    ✅ STRONG discrimination by AlphaFold")
elif delta_mean > 10:
    print(f"    ⚠️  MODERATE discrimination by AlphaFold")
else:
    print(f"    ⚠️  LIMITED discrimination by AlphaFold")

print(f"\n🎯 QUANTUM COMPUTING:")
print(f"    Regions with pLDDT < 70 ideal for quantum sampling:")
print(f"    • Ubiquitin:        {high_conf['statistics']['medium_conf'] + high_conf['statistics']['low_conf']:.1f}%")
print(f"    • Alpha-synuclein:  {low_conf['statistics']['medium_conf'] + low_conf['statistics']['low_conf']:.1f}%")

## 📊 Visualization - 6-Panel Comparison Plot

In [ ]:
# Create comprehensive comparison plot
fig = plt.figure(figsize=(16, 10))

hc_plddt = high_conf['plddt']
hc_res = high_conf['residue_numbers']
hc_name = high_conf['protein_info']['name']
hc_stats = high_conf['statistics']

lc_plddt = low_conf['plddt']
lc_res = low_conf['residue_numbers']
lc_name = low_conf['protein_info']['name']
lc_stats = low_conf['statistics']

# Plot 1: High Confidence pLDDT along sequence
ax1 = plt.subplot(3, 2, 1)
ax1.plot(hc_res, hc_plddt, linewidth=2, color='#2E86AB')
ax1.axhline(y=90, color='green', linestyle='--', alpha=0.5, label='Very High (90)')
ax1.axhline(y=70, color='orange', linestyle='--', alpha=0.5, label='High (70)')
ax1.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='Medium (50)')
ax1.fill_between(hc_res, hc_plddt, alpha=0.3, color='#2E86AB')
ax1.set_xlabel('Residue Number', fontsize=11)
ax1.set_ylabel('pLDDT Score', fontsize=11)
ax1.set_title(f'{hc_name}\nHigh Confidence Protein', fontsize=12, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(alpha=0.3)
ax1.set_ylim([0, 105])

# Plot 2: Low Confidence pLDDT along sequence
ax2 = plt.subplot(3, 2, 2)
ax2.plot(lc_res, lc_plddt, linewidth=2, color='#A23B72')
ax2.axhline(y=90, color='green', linestyle='--', alpha=0.5, label='Very High (90)')
ax2.axhline(y=70, color='orange', linestyle='--', alpha=0.5, label='High (70)')
ax2.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='Medium (50)')
ax2.fill_between(lc_res, lc_plddt, alpha=0.3, color='#A23B72')
ax2.set_xlabel('Residue Number', fontsize=11)
ax2.set_ylabel('pLDDT Score', fontsize=11)
ax2.set_title(f'{lc_name}\nIntrinsically Disordered Protein', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(alpha=0.3)
ax2.set_ylim([0, 105])

# Plot 3: Distribution histogram
ax3 = plt.subplot(3, 2, 3)
ax3.hist(hc_plddt, bins=30, alpha=0.6, label=hc_name, color='#2E86AB', edgecolor='black')
ax3.hist(lc_plddt, bins=30, alpha=0.6, label=lc_name, color='#A23B72', edgecolor='black')
ax3.axvline(x=90, color='green', linestyle='--', alpha=0.5)
ax3.axvline(x=70, color='orange', linestyle='--', alpha=0.5)
ax3.axvline(x=50, color='red', linestyle='--', alpha=0.5)
ax3.set_xlabel('pLDDT Score', fontsize=11)
ax3.set_ylabel('Frequency', fontsize=11)
ax3.set_title('pLDDT Distribution Comparison', fontsize=12, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(alpha=0.3)

# Plot 4: Box plots
ax4 = plt.subplot(3, 2, 4)
bp = ax4.boxplot([hc_plddt, lc_plddt], 
                 tick_labels=[hc_name[:20], lc_name[:20]],
                 patch_artist=True, showmeans=True)
bp['boxes'][0].set_facecolor('#2E86AB')
bp['boxes'][1].set_facecolor('#A23B72')
ax4.axhline(y=90, color='green', linestyle='--', alpha=0.5)
ax4.axhline(y=70, color='orange', linestyle='--', alpha=0.5)
ax4.axhline(y=50, color='red', linestyle='--', alpha=0.5)
ax4.set_ylabel('pLDDT Score', fontsize=11)
ax4.set_title('Distribution Statistics', fontsize=12, fontweight='bold')
ax4.grid(alpha=0.3)

# Plot 5: Confidence category bar chart
ax5 = plt.subplot(3, 2, 5)
categories = ['Very High\n(>90)', 'High\n(70-90)', 'Medium\n(50-70)', 'Low\n(<50)']
hc_values = [hc_stats['very_high_conf'], hc_stats['high_conf'],
             hc_stats['medium_conf'], hc_stats['low_conf']]
lc_values = [lc_stats['very_high_conf'], lc_stats['high_conf'],
             lc_stats['medium_conf'], lc_stats['low_conf']]

x = np.arange(len(categories))
width = 0.35

ax5.bar(x - width/2, hc_values, width, label=hc_name[:20], color='#2E86AB', alpha=0.8)
ax5.bar(x + width/2, lc_values, width, label=lc_name[:20], color='#A23B72', alpha=0.8)
ax5.set_xlabel('Confidence Category', fontsize=11)
ax5.set_ylabel('Percentage of Residues (%)', fontsize=11)
ax5.set_title('Confidence Category Distribution', fontsize=12, fontweight='bold')
ax5.set_xticks(x)
ax5.set_xticklabels(categories, fontsize=9)
ax5.legend(fontsize=10)
ax5.grid(alpha=0.3, axis='y')

# Plot 6: Summary table
ax6 = plt.subplot(3, 2, 6)
ax6.axis('off')

summary_data = [
    ['Metric', hc_name[:25], lc_name[:25], 'Difference'],
    ['Mean pLDDT', f"{hc_stats['mean']:.1f}", f"{lc_stats['mean']:.1f}",
     f"{hc_stats['mean']-lc_stats['mean']:.1f}"],
    ['Std Dev', f"{hc_stats['std']:.1f}", f"{lc_stats['std']:.1f}",
     f"{hc_stats['std']-lc_stats['std']:.1f}"],
    ['Min pLDDT', f"{hc_stats['min']:.1f}", f"{lc_stats['min']:.1f}",
     f"{hc_stats['min']-lc_stats['min']:.1f}"],
    ['Max pLDDT', f"{hc_stats['max']:.1f}", f"{lc_stats['max']:.1f}",
     f"{hc_stats['max']-lc_stats['max']:.1f}"],
    ['Very High %', f"{hc_stats['very_high_conf']:.1f}",
     f"{lc_stats['very_high_conf']:.1f}",
     f"{hc_stats['very_high_conf']-lc_stats['very_high_conf']:.1f}"],
    ['Confidence', high_conf['confidence'],
     low_conf['confidence'], '-'],
]

table = ax6.table(cellText=summary_data, cellLoc='center', loc='center',
                 colWidths=[0.25, 0.25, 0.25, 0.25])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)

# Style header row
for i in range(4):
    table[(0, i)].set_facecolor('#4A4A4A')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax6.set_title('Summary Statistics Comparison', fontsize=12, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\n✅ Visualization complete!")

## Save Results

In [ ]:
# Save as JSON
results = {
    'high_confidence': {
        'name': high_conf['protein_info']['name'],
        'uniprot_id': high_conf['protein_info']['uniprot_id'],
        'statistics': high_conf['statistics'],
        'confidence': high_conf['confidence'],
    },
    'low_confidence': {
        'name': low_conf['protein_info']['name'],
        'uniprot_id': low_conf['protein_info']['uniprot_id'],
        'statistics': low_conf['statistics'],
        'confidence': low_conf['confidence'],
    },
    'comparison': {
        'delta_mean_plddt': float(delta_mean),
        'mean_ratio': float(ratio)
    }
}

with open('alphafold_comparison_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("💾 Results saved to: alphafold_comparison_results.json")
print("\n" + "="*70)
print("✅ ANALYSIS COMPLETE!")
print("="*70)
print(f"\n📊 Summary:")
print(f"   • {HIGH_CONFIDENCE_PROTEIN['name']}: {high_conf['statistics']['mean']:.1f} pLDDT")
print(f"   • {LOW_CONFIDENCE_PROTEIN['name']}: {low_conf['statistics']['mean']:.1f} pLDDT")
print(f"   • Difference: {delta_mean:.1f} points")
print(f"\n🎯 AlphaFold successfully distinguishes structured vs disordered proteins!")